In [ ]:
# Sentiment Analysis Model Development

In [ ]:
## Loading

In [ ]:
df = pd.read_csv('eda_data.csv')

In [ ]:
# Function to create and return the Bag of Words DataFrame

In [ ]:
def create_bow_representation(texts, use_tfidf=False, max_features=100):

In [ ]:
    vectorizer = TfidfVectorizer(max_features=max_features) if use_tfidf else CountVectorizer(max_features=max_features)

In [ ]:
    X = vectorizer.fit_transform(texts)

In [ ]:
    feature_names = vectorizer.get_feature_names_out()

In [ ]:
    df_bow = pd.DataFrame(X.toarray(), columns=feature_names)

In [ ]:
    return df_bow, feature_names

In [ ]:
df_bow, feature_names = create_bow_representation(df['cleaned_text'], use_tfidf=False, max_features=50)

In [ ]:
def visualize_bow(df_bow, feature_names, annotate=False):

In [ ]:
    plt.figure(figsize=(12, 8))

In [ ]:
    sns.heatmap(df_bow, annot=annotate, cmap='viridis', cbar=False, xticklabels=feature_names)

In [ ]:
    plt.title('Bag of Words Matrix')

In [ ]:
    plt.xlabel('Features')

In [ ]:
    plt.ylabel('Documents')

In [ ]:
    plt.show()

In [ ]:
visualize_bow(df_bow, feature_names, annotate=False)

In [ ]:
# Logistic Regression Model

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

In [ ]:
vectorizer = TfidfVectorizer()

In [ ]:
X = vectorizer.fit_transform(df['cleaned_text'])

In [ ]:
y = df['class']

In [ ]:
encoder = LabelEncoder()

In [ ]:
y_encoded = encoder.fit_transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=35476648)

In [ ]:
model = LogisticRegression(max_iter=1000)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
print("Classification Report:")

In [ ]:
print(classification_report(y_test, predictions))

In [ ]:
probabilities = model.predict_proba(X_test)

In [ ]:
auc_score = roc_auc_score(y_test, probabilities, multi_class='ovr')

In [ ]:
print(f"ROC AUC Score: {auc_score:.2f}")

In [ ]:
# SVM Model

In [ ]:
from sklearn.svm import SVC

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,2))

In [ ]:
X = vectorizer.fit_transform(df['cleaned_text'])

In [ ]:
y = df['class']

In [ ]:
encoder = LabelEncoder()

In [ ]:
y_encoded = encoder.fit_transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=35476648)

In [ ]:
model = SVC(class_weight='balanced', probability=True)

In [ ]:
param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}

In [ ]:
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1_macro', verbose=1)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
best_model = grid_search.best_estimator_

In [ ]:
predictions = best_model.predict(X_test)

In [ ]:
probs = best_model.predict_proba(X_test)

In [ ]:
auc_score = roc_auc_score(y_test, probs, multi_class='ovr')

In [ ]:
print("Classification Report:")

In [ ]:
print(classification_report(y_test, predictions))

In [ ]:
print(f"ROC AUC Score: {auc_score:.2f}")

In [ ]:
# Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
vectorizer = TfidfVectorizer(max_features=1000)

In [ ]:
X = vectorizer.fit_transform(df['cleaned_text'])

In [ ]:
y = df['class']

In [ ]:
encoder = LabelEncoder()

In [ ]:
y_encoded = encoder.fit_transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=35476648)

In [ ]:
rf = RandomForestClassifier(random_state=42, class_weight='balanced')

In [ ]:
param_grid = {

In [ ]:
    'n_estimators': [100, 200, 300],

In [ ]:
    'max_depth': [10, 20, 30],

In [ ]:
    'min_samples_split': [2, 5, 10],

In [ ]:
    'min_samples_leaf': [1, 2, 4]

In [ ]:
}

In [ ]:
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
best_model = grid_search.best_estimator_

In [ ]:
predictions = best_model.predict(X_test)

In [ ]:
probs = best_model.predict_proba(X_test)